In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic
from datetime import datetime, timedelta
from anthropic.types import Message, ToolParam
import json

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [2]:
# Helper functions
def add_user_message(
    messages: list[Message | str], 
    message: Message | str
    ):
    """
    Add a user message to the list of messages.

    Args:
        messages (list[Message  |  str]): The list of messages.
        message (Message | str): The message to add.
    """
    user_message = {
        "role": "user", 
        "content": message.content if isinstance(message, Message) else message
        }
    messages.append(user_message)


def add_assistant_message(
    messages: list[Message | str], 
    message: Message | str
    ):
    """
    Add an assistant message to the list of messages.
    
    Args:
        messages (list[Message | str]): The list of messages.
        message (Message | str): The message to add.
    """ 
    assistant_message = {"role": "assistant", "content": message.content if isinstance(message, Message) else message}
    messages.append(assistant_message)


def chat(
    messages: list[Message | str], 
    system: str | None = None, 
    temperature: float = 1.0, 
    stop_sequences: list[str] = [],
    tools: list[ToolParam] | None = None
    ) -> Message:
    """
    Chat with the model.
    
    Args:
        messages (list[Message | str]): The list of messages.
        system (str | None): The system message.
        temperature (float): The temperature.
        stop_sequences (list[str]): List of stop sequences.
        tools (list[ToolParam] | None): List of tools.
        
    Returns:
        Message: The message.
    """
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system
        
    if tools:
        params["tools"] = tools

    message = client.messages.create(**params)
    return message

def text_from_message(message: Message) -> str:
    """
    Get the text from a message.
    
    Args:
        message (Message): The message.
    """
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [3]:
# Tools and Schemas

def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
    if not date_format:
        raise ValueError("date_format cannot be empty")
    return datetime.now().strftime(date_format)


def add_duration_to_datetime(
    datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"
):
    date = datetime.strptime(datetime_str, input_format)

    if unit == "seconds":
        new_date = date + timedelta(seconds=duration)
    elif unit == "minutes":
        new_date = date + timedelta(minutes=duration)
    elif unit == "hours":
        new_date = date + timedelta(hours=duration)
    elif unit == "days":
        new_date = date + timedelta(days=duration)
    elif unit == "weeks":
        new_date = date + timedelta(weeks=duration)
    elif unit == "months":
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        if month == 0:
            month = 12
            year -= 1
        day = min(
            date.day,
            [
                31,
                29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == "years":
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")


def set_reminder(content, timestamp):
    print(f"----\nSetting the following reminder for {timestamp}:\n{content}\n----")
    
    
get_current_datetime_schema = ToolParam({
    "name": "get_current_datetime",
    "description": "Returns the current date and time formatted according to the specified format",
    "input_schema": {
        "type": "object",
        "properties": {
            "date_format": {
                "type": "string",
                "description": "A string specifying the format of the returned datetime. Uses Python's strftime format codes.",
                "default": "%Y-%m-%d %H:%M:%S"
            }
        },
        "required": []
    }
})


add_duration_to_datetime_schema = {
    "name": "add_duration_to_datetime",
    "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
    "input_schema": {
        "type": "object",
        "properties": {
            "datetime_str": {
                "type": "string",
                "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
            },
            "duration": {
                "type": "number",
                "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
            },
            "unit": {
                "type": "string",
                "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
            },
            "input_format": {
                "type": "string",
                "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
            },
        },
        "required": ["datetime_str"],
    },
}

set_reminder_schema = {
    "name": "set_reminder",
    "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
    "input_schema": {
        "type": "object",
        "properties": {
            "content": {
                "type": "string",
                "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
            },
            "timestamp": {
                "type": "string",
                "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
            },
        },
        "required": ["content", "timestamp"],
    },
}

batch_tool_schema = {
    "name": "batch_tool",
    "description": "Invoke multiple other tool calls simultaneously",
    "input_schema": {
        "type": "object",
        "properties": {
            "invocations": {
                "type": "array",
                "description": "The tool calls to invoke",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the tool to invoke",
                        },
                        "arguments": {
                            "type": "string",
                            "description": "The arguments to the tool, encoded as a JSON string",
                        },
                    },
                    "required": ["name", "arguments"],
                },
            }
        },
        "required": ["invocations"],
    },
}

pass

In [4]:
messages = []
messages.append({
    "role": "user",
    "content": "What is the exact time, formatted as HH:MM:SS?"
})

response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[get_current_datetime_schema],
)

# append multiblock message
messages.append({
    "role": "assistant",
    "content": response.content
})

messages

[{'role': 'user', 'content': 'What is the exact time, formatted as HH:MM:SS?'},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='toolu_01Rbn5JPiHxch5HEsePCjBhG', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]}]

In [5]:
# tool_call = messages[1]['content'][0] # ToolUseBlock
# tool_call


In [6]:
# print(tool_call.id)
# print(tool_call.name)
# print(tool_call.input)
# print(tool_call.type)






In [7]:
# ToolResultBlock includes the output of the tool call
# The tool result block has several important properties:

# tool_use_id - Must match the id of the ToolUse block that this ToolResult corresponds to
# content - Output from running your tool, serialized as a string
# is_error - True if an error occurred

last_message = messages[-1]
input = last_message['content'][0].input
current_time = get_current_datetime(**input)
current_time

messages.append({
    "role": "user",
    "content": [{
        # ToolResultBlock
        "type": "tool_result",
        "tool_use_id": last_message['content'][0].id,
        "content": current_time,
        "is_error": False
    }]
})

messages



[{'role': 'user', 'content': 'What is the exact time, formatted as HH:MM:SS?'},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='toolu_01Rbn5JPiHxch5HEsePCjBhG', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01Rbn5JPiHxch5HEsePCjBhG',
    'content': '15:06:58',
    'is_error': False}]}]

In [8]:
response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[get_current_datetime_schema]
)

response

Message(id='msg_01R5ETKc6ot6VTX8ttq4ofLN', content=[TextBlock(citations=None, text='The exact time is **15:06:58** (3:06:58 PM).', type='text')], model='claude-haiku-4-5-20251001', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=699, output_tokens=23, server_tool_use=None, service_tier='standard', inference_geo='not_available'))

In [9]:
def run_tool(tool_name: str, tool_input: dict) -> str:
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    elif tool_name == "add_duration_to_datetime":
        return add_duration_to_datetime(**tool_input)
    elif tool_name == "set_reminder":
        return set_reminder(**tool_input)
    else:
        raise ValueError(f"Unsupported tool: {tool_name}")

def run_tools(message: Message) -> list[dict]:
    """
    Run the tools.
    
    Args:
        message (Message): The message.
        
    Returns:
        list[dict]: The tool result blocks.
    """
    
    tool_requests = [
        block for block in message.content if block.type == "tool_use"
    ]
    
    # ToolResultBlock definition (response to Claude's request to run a tool)
    # tool_result_block = {
    # "type": "tool_result",
    # "tool_use_id": tool_request.id,
    # "content": json.dumps(tool_output),
    # "is_error": False
    # }
    
    
    tool_result_blocks = []
    
    for tool_request in tool_requests:
        # Process each tool request...
        try:
            tool_output = run_tool(tool_request.name, tool_request.input)
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False
            }
            tool_result_blocks.append(tool_result_block)
        except Exception as e:
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True
            }
            tool_result_blocks.append(tool_result_block)        
        
    return tool_result_blocks
            

def run_conversation(messages: list[Message | str]) -> list[Message | str]:
    """
    Run the conversation.
    
    Args:
        messages (list[Message | str]): The messages.
        
    Returns:
        list[Message | str]: The messages.
        
    """
    while True:
        response = chat(
            messages, 
            tools=[get_current_datetime_schema, add_duration_to_datetime_schema, set_reminder_schema]
            )
        add_assistant_message(messages, response)
        print(text_from_message(response))
        
        # Claude issues a stop_reason when it wants to use a tool
        # So, the loop continues until the stop_reason is not "tool_use"
        if response.stop_reason != "tool_use":
            break
            
        # if the stop_reason is "tool_use", run the tools
        tool_results = run_tools(response)
        add_user_message(messages, tool_results)
    
    return messages

In [10]:
# multiple turns with tool calls

messages = []

#add_user_message(messages, "What is the current time in HH:MM format? Also, what is the current time in SS format?")

add_user_message(messages, "Set a reminder for my doctor's appointment. The appointment is 177 days after January 1, 2050.")
run_conversation(messages)

I need to calculate when 177 days after January 1, 2050 is, and then set a reminder for that date.
Now I'll set a reminder for your doctor's appointment on that date:
----
Setting the following reminder for 2050-06-27T00:00:00:
Doctor's appointment
----
Perfect! I've set a reminder for your doctor's appointment on **Monday, June 27, 2050** at 12:00 AM. This is 177 days after January 1, 2050. You'll receive a notification at that time.


[{'role': 'user',
  'content': "Set a reminder for my doctor's appointment. The appointment is 177 days after January 1, 2050."},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text='I need to calculate when 177 days after January 1, 2050 is, and then set a reminder for that date.', type='text'),
   ToolUseBlock(id='toolu_015qdP4LEvWYbAANfGJa7EvC', input={'datetime_str': '2050-01-01', 'duration': 177, 'unit': 'days', 'input_format': '%Y-%m-%d'}, name='add_duration_to_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_015qdP4LEvWYbAANfGJa7EvC',
    'content': '"Monday, June 27, 2050 12:00:00 AM"',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="Now I'll set a reminder for your doctor's appointment on that date:", type='text'),
   ToolUseBlock(id='toolu_01HckCLZXdUG54LxJAX1dNzG', input={'content': "Doctor's appointment", 'timestamp': '2050-06-27T00:00:00'}, name='s